In [42]:
import pandas as pd
pd.options.display.float_format = '{:,.2f}'.format
from constantes import WINSIG_SERVICIO_FARMACIA_CC_SIGCOM

In [43]:
def desglosar_centro_de_costo(desglose, total_dinero):
    con_dinero = desglose.copy()
    con_dinero['TOTAL_X_PORCENTAJE'] = con_dinero['PORCENTAJES'] * total_dinero

    return con_dinero


In [44]:
df_winsig = pd.read_excel('input\\Informe Winsig Septiembre 2022.xlsx', header=7)
df_winsig = df_winsig.dropna(axis=1, how='all').dropna(axis=0, how='all')
archivo_produccion = pd.ExcelFile('input\\output_producciones.xlsx')


In [55]:
desglose_pabellon = pd.read_excel(archivo_produccion, sheet_name='PABELLÓN')
total_pabellon = df_winsig.query('SERVICIO == "B. PABELLON"')['Gasto Servicio'].iloc[0]
con_gastos_pabellon = desglosar_centro_de_costo(desglose_pabellon, total_pabellon)

In [56]:
desglose_policlinico = pd.read_excel(archivo_produccion, sheet_name='CONSULTAS SIN MANEJO DEL DOLOR')
total_policlinico = df_winsig.query('SERVICIO == "POLICLÍNICO"')['Gasto Servicio'].iloc[0]
con_gastos_policlinico = desglosar_centro_de_costo(desglose_policlinico, total_policlinico)


In [65]:
con_gastos_pabellon_para_concatenar = con_gastos_pabellon.iloc[:, [0, 1, 2, -1, 3]]
con_gastos_pabellon_para_concatenar.iloc[:, [1, 2, -1]] = None
con_gastos_pabellon_para_concatenar.columns = df_winsig.columns

In [68]:
con_gastos_policlinico_para_concatenar = con_gastos_policlinico.iloc[:, [0, 1, 2, -1, 3]]
con_gastos_policlinico_para_concatenar.iloc[:, [1, 2, -1]] = None
con_gastos_policlinico_para_concatenar.columns = df_winsig.columns

In [71]:
winsig_concatenado = df_winsig.copy()
winsig_mas_desglose = pd.concat([winsig_concatenado, con_gastos_pabellon_para_concatenar])
winsig_mas_desglose = pd.concat([winsig_mas_desglose, con_gastos_policlinico_para_concatenar])

In [73]:
servicios_a_sacar = ['B. PABELLON', 'PABELLÓN', 'POLICLÍNICO', 'CONSULTAS SIN MANEJO DEL DOLOR']

In [76]:
servicios_a_dejar = ~(winsig_mas_desglose['SERVICIO'].isin(servicios_a_sacar))

In [78]:
winsig_final = winsig_mas_desglose[servicios_a_dejar]

In [79]:
winsig_final

,SERVICIO,recetas,prescripciones,Gasto Servicio,Costo por receta
0,HMQ RESPIRATORIO SALA,718.00,"2,929.00","70,311,807.00","97,927.31"
1,INTERMEDIO INDIFERENCIADO 1 [4°Piso Sector Nor...,625.00,"3,248.00","1,893,404.00","3,029.45"
2,INTERMEDIO INDIFERENCIADO 2 [4°Piso Sector Sur...,687.00,"3,010.00","27,424,336.00","39,918.98"
3,INTERMEDIO INDIFERENCIADO 3 [5°Piso Sector Norte],433.00,"2,225.00","6,309,533.00","14,571.67"
4,MÉDICO QUIRURGICO CARDIOVASCULAR[4°Piso Sector...,0.00,0.00,0.00,NaN
5,QUIMIOTERAPIA INT [3 Piso Sur],0.00,0.00,0.00,NaN
6,ELECTROFISIOLOGÍA,4.00,15.00,"113,294.00","28,323.50"
7,HEMODINAMIA,12.00,23.00,"1,930,656.00","160,888.00"
8,BRONCOSCOSPIA,80.00,83.00,"105,884.00","1,323.55"
9,UCI,700.00,"3,002.00","20,797,767.00","29,711.10"
